### Question 9

In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import numpy.linalg as npla
import scipy.linalg as spla

In [2]:
def moving(i,j,action):
    # action direction : upwards=2, downwards=3, left=1, right=0
    newi = 0
    newj = 0
    # moving into terminating state
    if i==0 and j==1 and action==1:
        newi = 0
        newj = 0
    elif i==1 and j==0 and action==2:
        newi = 0
        newj = 0
    elif i==2 and j==3 and action==3:
        newi = 3
        newj = 3
    elif i==3 and j==2 and action==0:
        newi = 3
        newj = 3
    #corners
    # at row=0, cannot move upwards
    elif i==0 and action==2:
        newi = i
        newj = j
    # at row=4, cannot move downwards
    elif i==3 and action==3:
        newi = i
        newj = j
    # at column=0, cannot move left
    elif j==0 and action==1:
        newi = i
        newj = j
    # at column=4, cannot move right
    elif j==3 and action==0:
        newi = i
        newj = j
    else:
        # move right
        if action==0:
            newi = i
            newj = j+1
        # move left
        elif action==1:
            newi = i
            newj = j-1
        # move upwards
        elif action==2:
            newi = i-1
            newj = j
        # move downwards
        elif action==3:
            newi = i+1
            newj = j
            
    return newi, newj, -1

In [3]:
# Initializing number of states, number of actions, gridSize and gamma
num_states = 14
num_actions = 4
gridSize = 4
threshold = pow(10,-13)

# vpi represents the value function
# piopt represents the policy
vpi = np.zeros((gridSize,gridSize),dtype='float')
piopt = np.ones((gridSize,gridSize,num_actions))
piopt /= float(num_actions)

iteration = 1
while True:
    print("Iteration: "+str(iteration))
    while True:
        delta = 0
        for i in range(0,gridSize):
            for j in range(0,gridSize):
                # if in terminating state continue
                if i==0 and j==0:
                    continue
                if i==3 and j==3:
                    continue
                # newValFun represents summation over all actions, 
                # p(s',r|s,a)*(reward received on choosing action and moving to state s' + vpi[s'])
                newValFun = 0.
                currValFun = vpi[i,j]
                for action in range(0,num_actions):
                    # new state (newi,newj) obtained after moving from state (i,j) under action and reward represents the 
                    # corresponding reward
                    newi,newj,reward = moving(i,j,action)
                    newValFun += piopt[i,j,action] * (reward + vpi[newi,newj])
                vpi[i,j] = newValFun

                delta = max(delta, abs(currValFun - newValFun))
        if delta<threshold:
            break
            
    print("State-Value Function: ")
    print(vpi)
    
    policy_stable = 1
    for i in range(0,gridSize):
        for j in range(0,gridSize):
            # if in terminating state continue
            if i==0 and j==0:
                continue
            if i==3 and j==3:
                continue
            currAction = np.random.choice(num_actions,p=piopt[i,j])
            # expReturn for each action, represents reward received on choosing action and moving to state s' + vpi[s']
            expReturn = np.zeros(num_actions)
            for action in range(0,num_actions):
                newi,newj,reward = moving(i,j,action)
                expReturn[action] = reward + vpi[newi,newj]
            optAction = np.argmax(expReturn)
            for k in range(0,4):
                piopt[i,j,k] = 0
            piopt[i,j,optAction] = 1
            if currAction != optAction:
                policy_stable = 0
    print("Policy: ")
    print(np.argmax(piopt, axis=2))
    if policy_stable==1:
        break
    iteration += 1

print("Optimal state-value function: ")
print(vpi)
print("Optimal policy: ")
print(np.argmax(piopt, axis=2))

Iteration: 1
State-Value Function: 
[[  0. -14. -20. -22.]
 [-14. -18. -20. -20.]
 [-20. -20. -18. -14.]
 [-22. -20. -14.   0.]]
Policy: 
[[0 1 1 1]
 [2 1 1 3]
 [2 2 0 3]
 [2 0 0 0]]
Iteration: 2
State-Value Function: 
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]
Policy: 
[[0 1 1 1]
 [2 1 0 3]
 [2 0 0 3]
 [0 0 0 0]]
Iteration: 3
State-Value Function: 
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]
Policy: 
[[0 1 1 1]
 [2 1 0 3]
 [2 0 0 3]
 [0 0 0 0]]
Optimal state-value function: 
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]
Optimal policy: 
[[0 1 1 1]
 [2 1 0 3]
 [2 0 0 3]
 [0 0 0 0]]


In [4]:
# Initializing number of states, number of actions, gridSize and gamma
num_states = 14
num_actions = 4
gridSize = 4
threshold = pow(10,-13)

# vpi represents the value function
# piopt represents the policy
vpi = np.zeros((gridSize,gridSize))
piopt = np.zeros((gridSize,gridSize))

iteration = 1
while True:
    print("Iteration: "+str(iteration))
    delta = 0
    
    for i in range(0,gridSize):
        for j in range(0,gridSize):
            # if in terminating state continue
            if i==0 and j==0:
                continue
            if i==3 and j==3:
                continue
            currValFun = vpi[i,j]
            # maxcoeff represents maximum over all actions, reward obtained from choosing action to get new state s'
            # + vpi(s')
            maxcoeff = -1000.
            for action in range(0,num_actions):
                newi,newj,reward = moving(i,j,action)
                tempstorage = reward + vpi[newi,newj]
                maxcoeff = max(maxcoeff, tempstorage)
            vpi[i,j] = maxcoeff
            delta = max(delta, abs(currValFun - maxcoeff))
            
    print("State-Value Function: ")
    print(vpi)
    if delta < threshold :
        break
    iteration += 1
    
for i in range(0,gridSize):
    for j in range(0,gridSize):
        # if in terminating state continue
        if i==0 and j==0:
            continue
        if i==3 and j==3:
            continue
        # expReturn for each action, represents reward received on choosing action and moving to state s' + vpi[s']
        expReturn = np.zeros(num_actions)
        for action in range(0,num_actions):
            newi,newj,reward = moving(i,j,action)
            expReturn[action] = reward + vpi[newi,newj]
        piopt[i,j] = np.argmax(expReturn)
        
print("Optimal state-value function: ")
print(vpi)
print("Optimal policy: ")
print(piopt)

Iteration: 1
State-Value Function: 
[[ 0. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1.  0.]]
Iteration: 2
State-Value Function: 
[[ 0. -1. -2. -2.]
 [-1. -2. -2. -2.]
 [-2. -2. -2. -1.]
 [-2. -2. -1.  0.]]
Iteration: 3
State-Value Function: 
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]
Iteration: 4
State-Value Function: 
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]
Optimal state-value function: 
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]
Optimal policy: 
[[0. 1. 1. 1.]
 [2. 1. 0. 3.]
 [2. 0. 0. 3.]
 [0. 0. 0. 0.]]


Since argmax resolves ties by selecting the smallest action (if action 1 and 3 both are maximizing policy then argmax picks action 1), the bug mentioned in exercise 4.4 does not arise